✨This a Jupyter Notebook that allows you to run Python code interactively 🐍✨

## Getting Started
1. Click the **Select Kernel** button on the top right of the notebook.
2. Choose **Python environments** and select **Python 3.11.9**.

Run code in the notebook by clicking the play button on the left side of the code cells.

## Learning Outcomes
We will focus on four key outcomes:

1. [Understanding agents and prompt engineering with Prompty.](#1-understanding-agents-and-prompt-engineering-with-prompty)
2. [Utilizing Prompty tracing for debugging and observability.](#2-utilizing-prompty-tracing-for-debugging-and-observabilty)
3. [Building and running Contoso Creative Writer.](#3-building-and-running-contoso-creative-writer)
4. [Setting up automated evaluations with GitHub Actions.](#4-setting-up-automated-evaluations-and-deployment-with-github-actions)

Let’s get started!

## 1. Understanding Agents and Prompt Engineering with Prompty
### 1.1. What are AI agents?
Contoso Creative Writer is an Agentic Application. 

**In artificial intelligence an agent is a program designed to:**

- Perceive its environment
- Make decisions
- Take actions to achieve specific goals

For Contoso Creative Writer, the goal is to help the marketing team at Contoso Outdoors write well-researched, product-specific articles. 
<br>Contoso Creative Writer is made up of 4 agents that help achieve this goal. 

<img src="../../images/agents.png" alt="Agents in Contoso Creative Writer" width="900" height="380">

### 1.2. How is an AI agent built?

Each agent in Contoso Creative Writer is built with [Prompty](https://prompty.ai/)! 

#### 1.2.1 What is Prompty?
Prompty is a new asset class and file format for LLM prompts that aims to provide observability, understandability, and portability for developers.

**The tooling for Prompty comes together in three ways:**

**i. A prompty file:**
- A Prompty file is not tied to any language as it uses the markdown format with YAML
- The file contains two main parts:
    - **Front Mattter:** 
        - This is the first part of the prompty file 
        - It is written in YAML and is contained inside two `---` seperators. 
        - It includes basic details about the prompt, the model configuration and prompty inputs. 

    - **Prompt Template:** 
        - This is the base prompt that is sent to the LLM once the prompty is executed. 
        - It uses Jinja format to pass values either specified in the front matter or from the application to the LLM.
        - Given *'name': Marlene*, the variable *{{name}}* will be replaced by *Marlene* at runtime. 

**ii. The VS Code extension tool:**
- The Prompty extension allows you to run Prompty files directly in VS Code. 
- It has been pre-installed for this workshop, but you can also find it in the Visual Studio Code Marketplace.

**iii. Runtimes in multiple programming languages:**
To execute a Prompty file asset in code, you can use one of the supporting runtimes such as:
- [Prompty Core (python)](https://github.com/microsoft/prompty/blob/main/runtime/prompty/README.md) - The python Prompty core package is the base package needed to run a prompty asset file.
- [Promptyflow (python)](https://microsoft.github.io/promptflow/tutorials/prompty-quickstart.html) - Use Prompty with Promptflow as your runtime and logic orchestrator.
- [LangChain (python)](https://github.com/langchain-ai/langchain/tree/master/libs/partners/prompty) - Use the experimental LangChain Prompty runtime.
- [Semantic Kernal (csharp)](https://www.developerscantina.com/p/semantic-kernel-prompty/) - Use the experimental Semantic Kernal Prompty runtime.

### 1.3. Building an AI Agent

To help us understand practically how we build an AI agent will build the **Researcher Agent** step by step.
<br>In order to build the Researcher agent you will complete the following 3 steps.

#### Steps to build the researcher agent
- [ ] Step 1: Build a multi-lingual query generator
- [ ] Step 2: Give the query generator a list of functions 
- [ ] Step 3: Build the tools and execute the research

##### **Step 1:** Build a multi-lingual query generator
- At it's core the researcher agent generates queries that can be used to look for information online. 
- It also allows us to find search results in multiple languages. 
- This can all be done using a single prompty file. 
- To see this in action open the [researcher-0.prompty](researcher/researcher-0.prompty) file and click the run button on the top right of the file. 
- The generated results from the LLM will be displayed in the *output* tab in the terminal. 

✅ To complete this step update the *sample* category in the front matter to change the instructions to use a new language.
<br> (For example use *es-ES* instead of *en-US*, to get the results back in Spanish) 

Alternatively to executing a Prompty file using the VS Code extension, here's how to execute it using the Prompty Python package:

In [ ]:
import prompty
import os
from IPython.display import Markdown

instructions = "Can you generate queries to find the latest camping trends and what folks are doing in the winter? Use 'en-US' as the market code. "
Markdown(prompty.execute(os.getcwd() + "/researcher/researcher-0.prompty", inputs={"instructions": instructions}))

##### **Step 2:** Give the query generator a list of functions 
- In order for the researcher to generate even better queries it needs to know which search functions are avaialble to it. 
- Prompty allows us to pass information to the LLM in the form of a json file using the *${file:filename.json}* format.
- In this step we want to give the LLM a list of the functions (sometimes called tools) that it can choose from.
- Open the [functions.json](researcher/functions.json) file. Read the descriptions of the **find_information**, **find_entities** and **find_news** functions. 
- Open the [researcher-1.prompty](researcher/researcher-1.prompty) file and note that *${file:functions.json}* has been added to *tools* under the *parameters* section in the front matter. 
- Click the run button on the top right of the file and look at the results in the output tab.

**Passing a json file to the *tools* parameter in Prompty invokes the LLM to:**
- return a list of dictionaries, where each dictionary represents a function to call and its arguments
- the *funcion* key contains the generated query and the selected market as arguments
- the *name* key contains the name of the most appropriate function to use

✅ To complete this step update the instructions passed in the *sample* section of the prompty file to influence which function the LLM chooses. 
<br>(For example to influence choice of the *find_news* function, use instructions: 'Can you find the latest news about Microsoft?')

##### **Step 3:** Understanding how the LLM chooses which function to call

**Let's walk through the code**

Let's import `execute_researcher_prompty` from the [researcher3.py](./researcher/researcher3.py) script:

In [16]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/researcher'))

from researcher3 import execute_researcher_prompty

The Prompty template is configured to load a catalog [functions.json](./researcher/functions.json) with the 3 functions:
- `find_information`
- `find_entities`
- `find_news`

Let's test each of one them to get a sense of how the LLM selects them

##### a.1. Find information

Let's see how the LLM can help us find information on the web. It cannot find information per se but it can select amongst a catalog of functions the one which would allow achieving that goal.

In [ ]:
instructions = "Can you find the best educational material for learning Python programming"
function_calls = execute_researcher_prompty(instructions=instructions)
function_calls

The `find_information` function was selected by the LLM based on the description of the function. It also figured out which parameter values should be passed to the function.

Go have a look at the definition of the `find_information` function in [functions.json](./researcher/functions.json) and try to get a sense of why this function was selected.

##### a.2. Find entities

In [ ]:
instructions = "Who is the inventor of the Python programming language?"
function_calls = execute_researcher_prompty(instructions=instructions)
function_calls

Is the `find_entities` function called? Why? Go look at the `find_entities` definition in [functions.json](./researcher/functions.json).

##### a.3. Find news

In [ ]:
instructions = "What's the latest about the Python programming language?"
function_calls = execute_researcher_prompty(instructions=instructions)
function_calls

🐞**BUG ALERT:** A bug has purposefully been left in the [functions.json](./researcher/functions.json) file.

Which function call has been selected? If any? With which parameters? Is it the `find_information` function we want?

**Exercice**:
- Find why the `find_news` function is not the one selected by the LLM
- Learn how to steer the LLM to select a given function using its description and parameters
- Add the function definition for `find_news` to the [functions.json](./researcher/functions.json) file
<details>
  <summary>find_news function definition</summary>
  
```json

  {
    "type": "function",
    "function": {
      "name": "find_news",
      "description": "Finds jokes on the web given a query. This function uses the Bing Search API to find jokes on the web given a query. The response includes the funniest jokes from the web and should be used if you're looking for a laugh.",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "An optimal search query to find jokes on the web using the Bing Search API"
          },
          "market": {
            "type": "string",
            "description": "The market to search in, e.g. en-US - it should match the language of the query"
          }
        },
        "required": [
          "query"
        ]
      }
    }
  }
```
</details>

##### **Step 4:** Build the functions and execute the research
- The researcher has now selected which function to use and has generated a query and market code to pass to it. 
- We now need to write the Python code for these functions that will pass the queries to the Bing Search API. 
- We will also use a function in Python to execute the prompty file, instead of running it manually in VS Code. 
- To put everything together click the play button on the left of the jupyter notebook code cell below below.  
- The notebook cell calls the *research* function to run code from **researcher3.py**.
- Open the the [researcher3.py](researcher/researcher3.py) file to see this code and try and understand what each function does. 

✅ To complete this step run the code in the cell below and test it out with different instructions!

In [ ]:
import sys
import os
from IPython.display import JSON

# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/researcher'))

from researcher3 import execute_function_calls

instructions = "Can you find the best educational material for learning Python programming"

# Execute the researcher prompty to get a list of functions calls
function_calls = execute_researcher_prompty(instructions=instructions)

# Execute the function calls
research = execute_function_calls(function_calls)
research

##### Congratulations you've build your first AI Agent with Prompty🎉
- [✅] Step 1: Build a multi-lingual query generator
- [✅] Step 2: Give the query generator a list of functions available
- [✅] Step 3: Build the tools and execute the research

We can now succesfully move on to learning outcome 2. 

## 2. Utilizing Prompty tracing for debugging and observabilty

When running Applications driven by LLMs, sometimes things don't go as expected! 
<br>It's important to have a way to debug your LLM workflow so you can see where things are working. 
<br>Tracing helps you visualize the execution of your prompts and clearly see what inputs are being passed to the LLM. 

To illustrate how to use local tracing in prompty let's build and debug a custom agent!

### 2.1 Run and debug a custom social media agent

<img src="../../images/socialmediaagent.png" alt="social media agent" width="600" height="350">

You should see a **socialmedia** folder in the workshop folder. This folder contains:

- [social.py](socialmedia/social.py) file: Uses the *execute_social_media_writer_prompty* and *run_social_media_agent* functions to send the inputs and prompts to the LLM. 
<br>It also imports the research function from the researcher agent to let it access information from online.  

- [social.prompty](socialmedia/social.prompty) file: This contains the base prompt for the social media agent. 
<br>It has been instructed to generate a thread of **4 tweets**.

🐞**BUG ALERT:** I have purposefully left a bug in the prompty file. We will use tracing to quickly spot the bug and fix it! 

##### Steps to build and debug the social media agent
- [ ] Step 1: Run the code for social media agent
- [ ] Step 2: Use Prompty tracing to identify and fix the bug


##### **Step 1:** Run the code for social media agent
- In order to run the code for social media agent click the play button on the left of the jupyter notebook code cell below. 
- What do you observe that is strange from the results?  

✅ To complete this step run the jupyter notebook cell below and try guess what's causing the bug. 

In [ ]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../docs/workshop/socialmedia'))

from social import run_social_media_agent

research_instructions = "Find information about AI Agents"
social_media_instructions = "Write a fun and engaging twitter thread about AI Agents given the research."

run_social_media_agent(instructions=research_instructions, social_media_instructions = social_media_instructions)

##### **Step 2:** Use Prompty tracing to identify and fix the bug
- To see the trace generated by Prompty open the **workshop** folder in the file explorer and look for a **.runs** folder in it. 
- Select this folder and click the **execute_social_media_writer_prompty.tracy** file at the top of the folder. 
- This file shows you information that prompty has sent to or recieved from the LLM. 
- In this specific case look at the *Completion Tokens* amount. This shows only 200 tokens, which is not enough for us to generate a thread on twitter. 
- Completion Tokens shows us what the number of tokens allocated for the LLM response, represented by the *max_tokens* parameter in the prompty file.
- To fix this bug go to the [social.prompty](socialmedia/social.prompty) file and edit the *max_tokens* amount to make it 1500.

✅ To complete this step **rerun the code in the cell above** and confirm the full thread is generated!
<br>(Note: If you get an error you may need to restart the notebook by clicking the *restart* button at the top of the notebook and then rerun the cell.)


##### Congratulations you've succesfully used Prompty tracing for debugging🎉
- [✅] Step 1: Run the code for social media agent
- [✅] Step 2: Use Prompty Tracing to identify and fix the bug

Now that we have a good understanding of how to build and debug agents with Prompty let's run Contoso Creative Writer, a multi-agent solution! 

## 3. Building and running Contoso Creative Writer 

#### Steps to build and run Contoso Creative Writer
- [ ] Step 1: Start the FastAPI server 
- [ ] Step 2: Start the Web Server
- [ ] Step 3: Test the app

**You will need to use the terminal to complete the steps:** 
- If it’s not already visible, you can open the terminal by clicking on the hamburger menu at the top left of the page 
- Click Terminal and select New Terminal.
- Copy and paste the commands in each step into the terminal and press enter to run them. 

##### **Step 1:** Start the FastAPI server 

1. We'll start by navigating to the correct folder

    Run the following command:

    ```bash
    cd ./src/api
    ```

2. Next we'll run the FastAPI webserver with the following command 

    ```bash
    fastapi dev main.py
    ```

3. Next you'll need to change the visibility of the **8000** and **5173** ports to public in the **PORTS** tab. 

    You can do this by right clicking on the visibility section of the port, selecting port visibility and setting it to public. 
    <br>The ports tab should look like this:

    <img src="../../images/ports.png" alt="Screenshot showing setting port-visibility" width="800px" />

✅ To complete this step once the ports are public navigate back to the terminal tab and confirm that you can see *Application startup complete*. 

##### **Step 2:** Start the web server

1. Open a **new terminal** 

    Once you've completed the above steps. You'll need to open a **new terminal** and navigate to the web folder. 
    <br>You can open a new terminal by clicking on the hamburger menu at the top left of the page, clicking Terminal and then selecting New Terminal. 

    In the terminal run the following commands 

    ```bash
    cd ./src/web
    ```
    
2. Next we will install node packages:

    ```bash
    npm install
    ```

3. Finally we can run the web app with a local dev web server:
    
    ```bash
    npm run dev
    ```

4. Navigate to the Application:
    - Once you've run the above command you should see an `http://localhost:5173/` link in the terminal. 
    - Right click the link or click the **open on browser** button that comes up as a Gitub notification in the bottom right corner of the screen. 
    - If you see a page from Github select the **continue** button. If not skip you should already see your app. 
    - You should now see the app appear on your screen!

✅ To complete this step confirm that you can see the Constoso Creative Writer application. 

##### **Step 3:** Test the app

- You can now test out the app by clicking **Example** to fill out the example information. Read the example instructions. 
- Click **Start Work** button to get Contoso Creative Writer to generate an article. 
- Click on the small **Debug** button at the bottom right of the Application to see which agent steps are carried out.

✅ To complete this step confirm that an article was generated that includes citation links and the products requested. 

##### Congratulations you've succesfully built and ran Contoso Creative Writer🎉
- [✅] Step 1: Start the FastAPI server 
- [✅] Step 2: Start the Web Server
- [✅] Step 3: Test the app

You've now almost completed the workshop, let's move on to the final learning outcome!

⏰**Note:** The next step takes a bit longer to complete: 
- Remember this lab is self paced so you can always complete it on your own, even after the session. 

## 4. Setting up automated evaluations and deployment with Github Actions 

Contoso Creative Writer is set up to run a CI/CD pipeline, which stands for Continuous Integration and Continuous Deployment. 

In this sample code the CI/CD pipeline includes the following: 
1. **Build and Deploy:** Automatically building and deploying the latest version of the code in production (This helps us confirm things are working as expected.)
2. **Evaulations:** Automatically running evaulations on a test article to see how fluent, grounded, relevant and coherent the final response was.

#### Steps to run automated evaluations and deployment 
- [ ] Step 1: Run azd up to deploy your application to production
- [ ] Step 2: Set up the azd pipeline configuration
- [ ] Step 3: Allow workflows in Github
- [ ] Step 4: Examine evaluations 

##### **Step 1:** Run azd up to deploy your application to production

In order to set up a CI/CD pipleline for this project we need to deploy our application to production. 

- Run the following command in the terminal to deploy the application:

    ```bash
    azd up
    ```
- This step will likely take several minutes. So feel free to take a break by stretching and walking around!
- Once complete scroll up in your terminal beyond any output from Github and look for *--- 🎉 | 4. Access your ACA deployed web app here:*
- This is a link to your deployed application. 
- While the subscription is active you can share this link with friends to show them what you've built! 

##### **Step 2:** Set up the azd pipeline configuration
We will set up the CI/CD pipeline with azd and GitHub actions. To do this **open a new terminal**. 

- Run the following command:

    ```shell
    azd pipeline config
    ```
- select an environment name like yourname-aitour
- select the recommended subscription by pressing enter
- select `Canada East` as the Azure Location 
- When asked to Log in using the Github CLI type in `Y`
- Choose `HTTPS` as the preferred protocol for Git Operations 
- Select `Y` to Authenticate with your Github credentials. 
- Choose Login with a web browser to authenticate Github CLI and follow the authentication instructions. 
- You may be asked if you want to commit and push your local changes. Choose `Y`
- You should see two links in your terminal. Select the sencond link to view your pipeline status. 

##### **Step 3:** Allow workflows in Github
- You will now see a screen in Github that tells you workflows are not being run on the fork. 
- Click the green button that says 'I understand my workflows go ahead and enable them.'
- If the screen is blank come back to the terminal and rerun the *azd pipeline config* command. 
- Again, select the sencond link to view your pipeline status. 
- You should now see the two Github action workflows running on the screen. 
- Look at the subheading of the actions and wait till the *Evaluate* one turns green. 
- It will likely take a few minutes to complete but once complete click the Evaulate action. 

##### **Step 4:** Examine evaluations

- You should see a table with some scores for relevance, fluencey, coherence and groundedness. 
- The scores are from 1-5, with 5 being the highest mark. These are used to help us know how well the model is performing.
- Examine the evaluations and think of some ways you might be able to improve a score in the future. 

##### Congratulations you've succesfully automated evaluations and deployment🎉
- [✅] Step 1: Run azd up to deploy your application to production
- [✅] Step 2: Set up the azd pipeline configuration
- [✅] Step 3: Allow workflows in Github
- [✅] Step 4: Examine evaluations 


## Conclusion

You have now completed the workshop! 

Today you learnt to:
1. Understand agents and prompt engineering with Prompty
2. Utilize Prompty tracing for debugging and observability
3. Build and run Contoso Creative Writer
4. Set up automated evaluations with GitHub Actions

We hope you enjoyed this workshop! You can continue to explore the code at [https://aka.ms/aitour/wrk551](https://aka.ms/aitour/wrk551).
<br>⭐Please star the [Github repo](https://aka.ms/aitour/wrk551) if you enjoyed this workshop⭐